In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yaml
import galsim

from pyimcom.config import Config, Settings as Stn
from pyimcom.coadd import Block
from pyimcom.analysis import OutImage, Mosaic
from concurrent.futures import ProcessPoolExecutor

In [2]:
#sys.path.append("/hildafs/projects/phy200017p/berlfein/metadetect-driver/")

In [3]:
from metadetect_driver import parse_driver_cfg, MetaDetectRunner, BaseRunner

In [4]:
## Default metadetect and driver configs
with open('../config/metadetect_default.yaml', 'r') as file:
    METADETECT_CONFIG= yaml.safe_load(file)
with open('../config/driver_default.yaml', 'r') as file:
    DEFAULT_DRIVER_CFG= yaml.safe_load(file)

## Generate Synthetic Data

### WCS
I am using one from the IMCOM coadds but substitute with whatever you like

In [6]:
# Read the config file from which the Mosaic was made
cfg = Config(cfg_file = '../IMCOM_data/configs/config_prod_H.json')
cfg.outstem = '../IMCOM_data/images/H158/output'
mosaic = Mosaic(cfg)
wcs = MetaDetectRunner.get_wcs(mosaic.outimages[0][1])

### Example image, psf, and other info

In [9]:
fwhm = 0.25
band = "H158"
pix_scale = 0.1
wcs = wcs
psf = galsim.Gaussian(fwhm=fwhm)

image = galsim.Gaussian(fwhm=1.0).drawImage(
            nx=500,
            ny=500,
            scale = pix_scale,
        ).array*1e2 + np.random.normal(0, 1e-2, (500, 500))

### Generate objects that contain image information
These are similar in name to the PyIMCOM objects to avoid changing much of the code. The `OutImage` object contains individual images,
while the `Mosaic` object contains a collection of images (from the same band).
The inputs:

- `image`: 2d numpy array containing your science image.
- `psf`: GalSim object with the PSF to use
- `wcs`: Astropy WCS object
- `pix_scale`: The pixel scale of your image (in arcsec/pix)
- `band`: The band name (defaults to None). If not added something might break later so just put something here for now.

#### Single image

In [10]:
## Set the "block" with the image
blk = BaseRunner.OutImage(image, psf, wcs, pix_scale, band)

In [11]:
# intialize the runner
bs_runner = BaseRunner.MetaDetectRunner(blk, meta_cfg = METADETECT_CONFIG, driver_cfg = DEFAULT_DRIVER_CFG)

In [12]:
# make catalog
res = bs_runner.make_catalog(save = False, save_blocks = False)

#### 2 images, different band

In [12]:
blk = BaseRunner.OutImage(image, psf, wcs, pix_scale, band)
blk2 = BaseRunner.OutImage(image, psf, wcs, pix_scale, band = 'J129')

In [13]:
bs_runner = BaseRunner.MetaDetectRunner([blk, blk2], meta_cfg = METADETECT_CONFIG, driver_cfg = DEFAULT_DRIVER_CFG)

In [17]:
res = bs_runner.make_catalog(save = False, save_blocks = False)

#### Multiple images, single band

In [14]:
mos = BaseRunner.Mosaic([blk, blk])
bs_runner = BaseRunner.MetaDetectRunner(mos, meta_cfg = METADETECT_CONFIG, driver_cfg = DEFAULT_DRIVER_CFG)

In [15]:
res = bs_runner.make_catalog(save = False, save_blocks = False)

In [30]:
#res

#### Multiple images, multiple bands

In [20]:
mos = BaseRunner.Mosaic([blk, blk])
mos2 = BaseRunner.Mosaic([blk2, blk2])
bs_runner = BaseRunner.MetaDetectRunner([mos, mos2], meta_cfg = METADETECT_CONFIG, driver_cfg = DEFAULT_DRIVER_CFG)

In [25]:
res = bs_runner.make_catalog(block_ids = [0,1], save = False, save_blocks = False)

In [31]:
#res